<a href="https://colab.research.google.com/github/meghanareddy712/CS384-Projects/blob/main/Fire_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 5.57 ms (started: 2022-04-23 12:50:27 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
# to read the images
from skimage.io import imread 
import tensorflow as tf
#groups layers into an object with training and inference features
from tensorflow.keras import models 
from tensorflow.keras import layers 
import sklearn

In [ ]:
!unzip '/content/drive/MyDrive/Design_Lab_Fire_Detection/dataset/fire/ds.zip'

In [ ]:
DATADIR = '/content/ds'
CATEGORIES = ['fire','smoke','neutral']


#iterete
for category in CATEGORIES:
  print(category)

In [ ]:
#iterete
images = []
test = []
train = []
train_label = []
test_label = []
run_test = []
for category in CATEGORIES:
  class_num = CATEGORIES.index(category) #Label the encoding
  path = os.path.join(DATADIR,category) #os lib used to create path to use all
  #print(path)
  i = 0
  n = len(os.listdir(path))
  for img in os.listdir(path):
      img_array = imread(os.path.join(path,img))
      try:
        img_resize = tf.image.resize(img_array,[250,250])
        if img_resize.shape[-1]!=3:
          continue
        if i==1:
          run_test.append(img_resize)
        if i<(0.7*n):
          train.append(img_resize)
          train_label.append(class_num)
        else:
          test.append(img_resize)
          test_label.append(class_num)
        i += 1
      except:
        continue     

In [ ]:
img_resize[-1].shape

In [ ]:
def plot_no_of_images(label):
  unique,count = np.unique(label, return_counts=True)
  print(unique,count)
  x = CATEGORIES
  y = count
  plt.bar(x,y,color='g')

  #plt.title('Gathered Data')
  plt.xlabel('CATEGORIES')
  plt.ylabel('Number of images')
  for i in range(len(x)):
    plt.text(i,y[i],y[i])

plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plot_no_of_images(train_label)
plt.title("Dataset gathered for training")
plt.subplot(1,2,2)
plot_no_of_images(test_label)
plt.title("Dataset gathered for testing")

In [ ]:
train_label = tf.one_hot(train_label,3) 
test_label = tf.one_hot(test_label,3)

image_test = tf.convert_to_tensor(test)
image_train = tf.convert_to_tensor(train)

image_train = (image_train).numpy() #tensor to numpy
image_test = (image_test).numpy()
train_label = train_label.numpy()
# Shuffling of data
image_train,image_label = sklearn.utils.shuffle(image_train,train_label)

image_test = tf.convert_to_tensor(image_test)
image_train = tf.convert_to_tensor(image_train)
image_label = tf.convert_to_tensor(image_label)
run_test = tf.convert_to_tensor(run_test)

In [ ]:
# image_train = (image_train/255).numpy() #tensor to numpy
# image_test = (image_test/255).numpy()

#CNN

In [ ]:
 # create the model
 cnn = models.Sequential([
     layers.Conv2D(filters=20, kernel_size=(3, 3),strides = 2, padding = 'same',activation='relu', input_shape=(250, 250, 3)), # in
     layers.MaxPooling2D((2, 2),strides = 2,padding = 'same'),
    
     layers.Conv2D(filters=20, kernel_size=(3, 3),strides = 2,padding = 'same', activation='relu'),
     layers.MaxPooling2D((2, 2),strides = 2,padding = 'same'),

     layers.Conv2D(filters=20, kernel_size=(3, 3),strides = 2,padding = 'same', activation='relu'),
     layers.MaxPooling2D((2, 2),strides = 2,padding = 'same'),
    
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
    
     layers.Dense(3, activation='softmax')# out
 ])

 # Compile the model
 cnn.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# Model summary
 cnn.summary()

In [ ]:
 callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
 history = cnn.fit(x = image_train, 
                    y = image_label, 
                   epochs = 10,
                   batch_size = 32,
                   validation_data = (image_test,test_label), 
                   callbacks=[callback]
                   )

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'],label = 'loss')
plt.plot(history.history['val_loss'],label = 'val_loss')
plt.title('Loss curve')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'],label = 'accuracy')
plt.plot(history.history['val_accuracy'],label = 'val_accuracy')
plt.title('Accuracy curve')
plt.legend()

In [ ]:
loss, acc = cnn.evaluate(image_test,test_label)
print(f"Training accuracy : {(acc*100):.2f} %")
print(f"Training loss : {loss:.4f}")

# Transfer Learning

In [ ]:
# Creating model from pretrained model : EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False
inputs = layers.Input(shape = (250, 250, 3), name = "input_layer")
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D(name = "global_average_pooling_2D")(x)
outputs = layers.Dense(3, activation = "softmax", name = "output_layer")(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss = "categorical_crossentropy",
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])

In [ ]:
#  Creating ModelCheckpoint callback
checkpoint_path = "modelCheckpoint.ckpt"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                          save_weights_only = True,
                                                          save_best_only = True,
                                                          save_frequency = "epoch",
                                                          verbose = 1)

In [ ]:
# Fit the model
initial_epoch = 5
history = model.fit(image_train,
                     image_label,
                     epochs = initial_epoch,
                     validation_data = (image_test, test_label),
                     callbacks = [checkpoint_callback]) 
model.save("Fire_smoke_model")

In [ ]:
# Fine-tuning model
base_model.trainable = True

for layer in base_model.layers[:-10]:
  layers.trainable = False

# Recompile model
model.compile(loss = "categorical_crossentropy", 
              optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
               metrics = ["accuracy"])

fine_tune_epochs = initial_epoch + 5

# Refit model
history_fine = model.fit(image_train,
                          image_label,
                          epochs = fine_tune_epochs,
                          validation_data = (image_test, test_label),
                          initial_epoch = history.epoch[-1])

In [ ]:
model.save("Fire_smoke_FineTuned")
base_model.summary()

In [ ]:
ypred = base_model.predict(image_test[1:])
ypred

Visualize training results
Create plots of loss and accuracy on the training and validation sets:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
y_pred = model.predict(image_test)
y_pred

#TESTING

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Design_Lab_Fire_Detection/Fire_smoke_FineTuned')

In [ ]:
# Check its architecture
new_model.summary()

In [ ]:
ypred = new_model.predict(run_test)
ypred

In [ ]:
ypred2 = new_model.predict(image_test)
ypred2

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(image_test, test_label, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

